<a href="https://colab.research.google.com/github/arishp/Automatic-License-Plate-Localization-and-Recognition-using-Pytesseract/blob/master/langchain_data_connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install langchain
# !pip install sentence-transformers
# !pip install chromadb
# !pip install langchain-community
# !pip install langchain-huggingface
# !pip install accelerate
# !pip install langchain-google-genai

In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_huggingface import HuggingFaceEndpoint

# Document Loaders

In [3]:
loader = TextLoader('sample.txt')
documents = loader.load()
len(documents)

1

# Document Transformers

In [4]:
text_splitter = CharacterTextSplitter(separator=" ", chunk_size=200, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
len(texts)

163

In [5]:
texts[:2]

[Document(metadata={'source': 'sample.txt'}, page_content='"Dhoni" redirects here. For other uses, see Dhoni (disambiguation).\nMahendra Singh Dhoni\nDhoni in 2023\nPersonal details\nBorn\t7 July 1981 (age 43)\nRanchi, Bihar (present-day Jharkhand), India\nHeight\t5'),
 Document(metadata={'source': 'sample.txt'}, page_content='ft 9 in (175 cm)[1]\nSpouse\tSakshi Dhoni\nAwards\t\n\n Padma Bhushan (2018)\n Padma Shri (2009)\n Major Dhyan Chand Khel Ratna Award (2008)\n\nNickname(s)\tMahi, Thala, Captain Cool[2]\nMilitary')]

# Text Embedding Models

In [6]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

# Vector Stores

In [7]:
db = Chroma.from_documents(texts, embeddings)

In [ ]:
db._collection.get(include=['embeddings'])

# Retrievers

In [9]:
retriever = db.as_retriever(search_kwargs={"k":3})

In [10]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f12ece5ac20>, search_kwargs={'k': 3})

# Question 1

In [21]:
question = input("enter your question?")

enter your question?who married dhoni?


In [23]:
docs = retriever.get_relevant_documents(question)

In [24]:
docs

[Document(metadata={'source': 'sample.txt'}, page_content='the field which has earned him the monicker "Captain cool".[178]\nPersonal life\n\nDhoni married Sakshi Singh Rawat on 4 July 2010 in Dehradun.[179][180] Dhoni and his wife have a daughter who was born'),
 Document(metadata={'source': 'sample.txt'}, page_content='Dhoni worked as a Travelling Ticket Examiner (TTE) at Kharagpur under South Eastern Railway zone of Indian Railways.[12][13]\nEarly career\n\nHe played as a wicket-keeper for Commando cricket club from'),
 Document(metadata={'source': 'sample.txt'}, page_content='on 6 February 2015.[181][182] He lives in his farmhouse outside Ranchi.[183] Dhoni is an automotive enthusiast and owns a number of bikes and cars in his collection.[184]\nCareer statistics\n\nDhoni has')]

In [43]:
relevant_text = ''
for doc in docs:
  relevant_text += doc.page_content
print(relevant_text)

the field which has earned him the monicker "Captain cool".[178]
Personal life

Dhoni married Sakshi Singh Rawat on 4 July 2010 in Dehradun.[179][180] Dhoni and his wife have a daughter who was bornDhoni worked as a Travelling Ticket Examiner (TTE) at Kharagpur under South Eastern Railway zone of Indian Railways.[12][13]
Early career

He played as a wicket-keeper for Commando cricket club fromon 6 February 2015.[181][182] He lives in his farmhouse outside Ranchi.[183] Dhoni is an automotive enthusiast and owns a number of bikes and cars in his collection.[184]
Career statistics

Dhoni has


In [50]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from google.colab import userdata

hf_token = userdata.get('HUGGINGFACE_HUB_API_KEY')

# Define the repository ID for the Gemma 2b model
repo_id = "google/gemma-2b"

# Set up a Hugging Face Endpoint for Gemma 2b model
llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=1024, temperature=0.1, huggingfacehub_api_token=hf_token
)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [67]:
template = """Answer the question in one line using the following information:

```{information}```.


*** Question ***

{question}

*** Answer ***"""

prompt = PromptTemplate.from_template(template=template)

In [68]:
prompt_formatted_str: str = prompt.format(question=question, information=relevant_text)
response = llm.invoke(prompt_formatted_str)
response

'\n\nDhoni married Sakshi Singh Rawat on 4 July 2010 in Dehradun.[179][180]'